In [1]:
import wandb
import os

wandb.login(key=os.environ.get("WANDB_API_KEY"))

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/sces213/.netrc
wandb: Currently logged in as: malaikanoor7864 (malaikanoor7864-mnsuam) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Initialize Weights and Biases
wandb.init(project="dnn-training", config={
    "learning_rate": 0.001,
    "batch_size": 32,
    "epochs": 50,
    "hidden_size": 128,
    "num_layers": 3
})

# Generate synthetic data with variance
torch.manual_seed(42)
num_samples = 1000
input_size = 10
output_size = 1

# Input data in range (-1, 1) with some variance
X = torch.rand(num_samples, input_size) * 2 - 1  # Range: (-1, 1)
y = torch.randn(num_samples, output_size) * 0.1 + X.mean(dim=1, keepdim=True)  # Add variance

# Create a dataset and dataloader
dataset = TensorDataset(X, y)
dataloader = DataLoader(dataset, batch_size=wandb.config.batch_size, shuffle=True)

# Define a deep neural network
class DNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(DNN, self).__init__()
        layers = []
        layers.append(nn.Linear(input_size, hidden_size))
        layers.append(nn.ReLU())
        
        for _ in range(num_layers - 1):
            layers.append(nn.Linear(hidden_size, hidden_size))
            layers.append(nn.ReLU())
        
        layers.append(nn.Linear(hidden_size, output_size))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

# Initialize the model, loss, and optimizer
model = DNN(input_size, wandb.config.hidden_size, output_size, wandb.config.num_layers)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=wandb.config.learning_rate)

# Watch the model with W&B
wandb.watch(model, log="all", log_freq=10)

# Training loop
for epoch in range(wandb.config.epochs):
    for batch_X, batch_y in dataloader:
        # Forward pass
        predictions = model(batch_X)
        loss = criterion(predictions, batch_y)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Log loss and gradients
        wandb.log({"loss": loss.item()})

    print(f"Epoch [{epoch+1}/{wandb.config.epochs}], Loss: {loss.item():.4f}")



wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch [1/50], Loss: 0.0193
Epoch [2/50], Loss: 0.0133
Epoch [3/50], Loss: 0.0089
Epoch [4/50], Loss: 0.0221
Epoch [5/50], Loss: 0.0057
Epoch [6/50], Loss: 0.0066
Epoch [7/50], Loss: 0.0162
Epoch [8/50], Loss: 0.0126
Epoch [9/50], Loss: 0.0174
Epoch [10/50], Loss: 0.0147
Epoch [11/50], Loss: 0.0034
Epoch [12/50], Loss: 0.0123
Epoch [13/50], Loss: 0.0062
Epoch [14/50], Loss: 0.0090
Epoch [15/50], Loss: 0.0050
Epoch [16/50], Loss: 0.0075
Epoch [17/50], Loss: 0.0021
Epoch [18/50], Loss: 0.0028
Epoch [19/50], Loss: 0.0033
Epoch [20/50], Loss: 0.0019
Epoch [21/50], Loss: 0.0037
Epoch [22/50], Loss: 0.0088
Epoch [23/50], Loss: 0.0033
Epoch [24/50], Loss: 0.0021
Epoch [25/50], Loss: 0.0056
Epoch [26/50], Loss: 0.0029
Epoch [27/50], Loss: 0.0028
Epoch [28/50], Loss: 0.0023
Epoch [29/50], Loss: 0.0010
Epoch [30/50], Loss: 0.0015
Epoch [31/50], Loss: 0.0017
Epoch [32/50], Loss: 0.0013
Epoch [33/50], Loss: 0.0040
Epoch [34/50], Loss: 0.0014
Epoch [35/50], Loss: 0.0006
Epoch [36/50], Loss: 0.0030
E